In [1]:
try:
    import geopandas as gpd
except ImportError:
    !pip install geopandas
    import geopandas as gpd
try:
    import netCDF4
except ImportError:
    !pip install netCDF4
    import netCDF4

import numpy as np
import pandas as pd
from shapely.geometry import Polygon, LineString, Point
import timeit
import datetime
import pickle

In [2]:
Regions = gpd.read_file("./Data/Weather/STATISTIK_AUSTRIA_GEM_20230101.shp", encoding='utf-8')
Regions.columns = ['id','name', 'geometry']
Regions['id'] = Regions['id'].astype(int)
Regions['bundesländer'] = pd.cut(Regions['id'], [10000, 20000,30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000], 
                                   labels = ['Burgenland','Kärnten', 'Niederösterreich', 'Oberösterreich', 'Salzburg', 'Steiermark', 'Tirol', 'Vorarlberg', 'Wien'])
Regions['bundesländer'] = Regions['bundesländer'].astype(str)
Regions['id'] = Regions['id'].astype(str)
Regions.rename(columns = {'id':'REGCODE', 'bundesländer':'BL'}, inplace = True)
Regions['REGCODE'] = Regions.REGCODE.apply(lambda x: "AT" + x)
Regions.head()

,REGCODE,name,geometry,BL
0,AT10101,Eisenstadt,"POLYGON ((640912.07 441878.1, 640860.94 441851...",Burgenland
1,AT10201,Rust,"POLYGON ((649520.23 441322.08, 649519.5 441310...",Burgenland
2,AT10301,Breitenbrunn am Neusiedler See,"POLYGON ((651359.33 458158.07, 651367.86 45815...",Burgenland
3,AT10302,Donnerskirchen,"POLYGON ((657987.56 448482.03, 657929.98 44847...",Burgenland
4,AT10303,Großhöflein,"POLYGON ((635646.04 445557.06, 635648.21 44553...",Burgenland


In [3]:
daily_rr = netCDF4.Dataset("./Data/Weather/SPARTACUS2-DAILY_COMBINED_2023_2024.nc")

In [4]:
# Load the dictionary from the pickle file
with open("./Data/Weather/new_regcode_to_pairs.pkl", "rb") as pickle_file:
    regcode_to_pairs = pickle.load(pickle_file)

time = daily_rr.variables['time'][:]
timedf = pd.DataFrame(time)
timedf.columns = ['timestamp']
timedf['date'] = pd.to_datetime(timedf["timestamp"].apply(lambda x:(datetime.datetime(1961, 1, 1) + datetime.timedelta(days=x)).date()))
# Extract week number
timedf['week_number'] = timedf['date'].dt.isocalendar().week
# Extract the month from the 'date' column
timedf['month'] = timedf['date'].dt.month

In [5]:
RR = daily_rr.variables['RR'][:]

In [ ]:
temp_dict = {}
for reg in Regions['REGCODE']:
    temp_dict[reg] = {}
    for time_idx in range(RR.shape[0]):
        rr_list= []  # Create an empty Pandas Series        
        for el in regcode_to_pairs[reg]:
            rr_list.append(RR[time_idx, el[0], el[1]])                
        temp_dict[reg][time_idx]={'max_value' : max(rr_list), 'percentile_95': np.percentile(rr_list, 95)}

/opt/conda/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4216: UserWarning: Warning: converting a masked element to nan.
  a = np.asanyarray(a)


In [ ]:
daily_count_dict = {}

for reg in Regions['REGCODE']:
    daily_count_dict[reg] = {}
    # Iterate through the data_dict and count days
    for key, values in temp_dict[reg].items():
        date = timedf.loc[key, 'date']
        daily_count_dict[reg][date] = values['max_value']

In [ ]:
with open("./Data/daily_count_dict.pkl", "wb") as file:
    pickle.dump(daily_count_dict, file)

In [ ]:
daily_count_dict["AT62209"]

In [ ]:
daily_count_dict["AT10305"]